In [1]:
#importing the csv file with the historical data to be used for backtesting
import pandas as pd
df = pd.read_csv("EURUSD_Candlestick_5_M_ASK_30.09.2019-30.09.2022.csv")
df

,Gmt time,Open,High,Low,Close,Volume
0,30.09.2019 00:00:00.000,1.09425,1.09426,1.09405,1.09406,585.10
1,30.09.2019 00:05:00.000,1.09408,1.09414,1.09401,1.09409,289.39
2,30.09.2019 00:10:00.000,1.09410,1.09423,1.09408,1.09410,276.24
3,30.09.2019 00:15:00.000,1.09409,1.09410,1.09388,1.09389,439.29
4,30.09.2019 00:20:00.000,1.09390,1.09395,1.09388,1.09395,341.23
...,...,...,...,...,...,...
225081,30.09.2022 20:35:00.000,0.98028,0.98034,0.98001,0.98022,624.12
225082,30.09.2022 20:40:00.000,0.98023,0.98047,0.98007,0.98030,408.20
225083,30.09.2022 20:45:00.000,0.98026,0.98034,0.98019,0.98031,317.29
225084,30.09.2022 20:50:00.000,0.98031,0.98067,0.97987,0.97999,1472.13


In [2]:
# reformatting the gmt time column to better suit the time format required removing the "000"
df["Gmt time"]=df["Gmt time"].str.replace(".000","")
# formatting the gmt time column some more into a datetime string we can use
df['Gmt time']=pd.to_datetime(df['Gmt time'], format='%d.%m.%Y %H:%M:%S')
# setting the gmt time column as the index of the rows in the excel file
df.set_index("Gmt time", inplace=True)
df=df[df.High!=df.Low]
len(df)

C:\Users\keith\AppData\Local\Temp/ipykernel_5452/1340908043.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Gmt time"]=df["Gmt time"].str.replace(".000","")


224989

## Adding technical indicators

In [3]:
import pandas_ta as ta
df["VWAP"]=ta.vwap(df.High, df.Low, df.Close, df.Volume)
# RSI Length and bbands length and std are values that can be experimented on 
# and more technical indicators can be added
df["RSI"]=ta.rsi(df.Close, length=16)
my_bbands = ta.bbands(df.Close, length=20, std=2.0)
df=df.join(my_bbands)

C:\Users\keith\AppData\Local\Temp/ipykernel_5452/1796816425.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["VWAP"]=ta.vwap(df.High, df.Low, df.Close, df.Volume)
C:\Users\keith\AppData\Local\Temp/ipykernel_5452/1796816425.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["RSI"]=ta.rsi(df.Close, length=16)


In [4]:
# Here we are computing the VWAP_Signal
# Here we compute the number of candles that are completely above or below the VWAP Curve 
VWAPSignal = [0]*len(df)
backcandles = 9

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df.Open[i], df.Close[i])>=df.VWAP[i]:
            dnt=0
        if min(df.Open[i], df.Close[i])<=df.VWAP[i]:
            upt=0
    if upt==1 and dnt==1:
        # No Change
        VWAPSignal[row]=3
    elif upt==1:
        # Uptrend
        VWAPSignal[row]=2
    elif dnt==1:
        # Downtrend
        VWAPSignal[row]=1

df['VWAPSignal'] = VWAPSignal

In [5]:
# Here we compute the total signal and determine where a buying or selling signal will be returned
def TotalSignal(l):
    if (df.VWAPSignal[l]==2
        and df.Close[l]<=df['BBL_20_2.0'][l]
        and df.RSI[l]<45):
            return 2
    if (df.VWAPSignal[l]==1
        and df.Close[l]>=df['BBU_20_2.0'][l]
        and df.RSI[l]>55):
            return 1
    return 0

TotSignal = [0]*len(df)
for row in range(backcandles, len(df)):
    TotSignal[row] = TotalSignal(row)
df['TotalSignal'] = TotSignal

In [6]:
df[df.TotalSignal!=0].count()

Open           3243
High           3243
Low            3243
Close          3243
Volume         3243
VWAP           3243
RSI            3243
BBL_20_2.0     3243
BBM_20_2.0     3243
BBU_20_2.0     3243
BBB_20_2.0     3243
BBP_20_2.0     3243
VWAPSignal     3243
TotalSignal    3243
dtype: int64

### Aids for visualizing the signals on the data 

In [7]:
# Creating points above or below wherever there are buying or selling signals 
import numpy as np
def pointposbreak(x):
    if x['TotalSignal']==1:
        return x['High']+1e-4
    elif x['TotalSignal']==2:
        return x['Low']-1e-4
    else:
        return np.nan

df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

In [8]:
# Plotting the candles, the indicators, the signals and the Bollinger bands to visualize what we have
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st = 10400
dfpl = df[st:st+350]
dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                 go.Scatter(x=dfpl.index, y=dfpl.VWAP, 
                           line=dict(color='blue', width=1), 
                           name="VWAP"), 
                go.Scatter(x=dfpl.index, y=dfpl['BBL_20_2.0'], 
                           line=dict(color='green', width=1), 
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_20_2.0'], 
                           line=dict(color='green', width=1), 
                           name="BBU")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="Signal")
fig.show()

### This is the backtesting portion of the code

In [9]:
dfpl = df[:225000].copy()
import pandas_ta as ta
dfpl['ATR']=ta.atr(dfpl.High, dfpl.Low, dfpl.Close, length=7)
#help(ta.atr)
def SIGNAL():
    return dfpl.TotalSignal

In [10]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.99
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 1.2*self.data.ATR[-1]
        TPSLRatio = 1.5

        if len(self.trades)>0:
            if self.trades[-1].is_long and self.data.RSI[-1]>=90:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
                self.trades[-1].close()
        
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100, margin=1/10, commission=0.00)
stat = bt.run()
stat

Start                     2019-09-30 00:00:00
End                       2022-09-30 20:55:00
Duration                   1096 days 20:55:00
Exposure Time [%]                     6.50654
Equity Final [$]                   337.307137
Equity Peak [$]                    346.994355
Return [%]                         237.307137
Buy & Hold Return [%]              -10.401623
Return (Ann.) [%]                   37.987446
Volatility (Ann.) [%]               20.602349
Sharpe Ratio                         1.843841
Sortino Ratio                        4.198914
Calmar Ratio                         3.151972
Max. Drawdown [%]                  -12.051962
Avg. Drawdown [%]                   -0.796937
Max. Drawdown Duration       68 days 06:25:00
Avg. Drawdown Duration        2 days 14:28:00
# Trades                                 2074
Win Rate [%]                        46.817743
Best Trade [%]                       0.436974
Worst Trade [%]                      -0.27659
Avg. Trade [%]                    

### Recent improvements 
- As of 07/01/2023, with a change to the period of the Bollinger Bands from a length of 14 to 20, as well as the adjustment of backcandles from 15 to 9, the following happened:
- Average annual return is up from 33% to 38%
- Return is up from 197% to 237% 
- Average trade is down from 51 minutes to 46 minutes
- Win rate is at 46%
        